In [1]:
import pandas as pd

In [88]:
import numpy as np

In [3]:
base_imaging_data = pd.read_csv('imaging_data.csv')
st_olav_36 = pd.read_csv('st_olav_36.csv')
freesurfer = pd.read_csv('freesurfer_and_bianca_results.csv')

In [48]:
st_olav_36.head()

,Unnamed: 0,subject-id,stolav_36,timepoint,Type of imaging
0,0,sub-400639,True,36-mon-follow-up,MR
1,1,sub-400460,True,36-mon-follow-up,MR
2,2,sub-400778,True,36-mon-follow-up,MR
3,3,sub-400698,True,36-mon-follow-up,MR
4,4,sub-400175,True,36-mon-follow-up,MR


In [47]:
st_olav_36['timepoint'] = '36-mon-follow-up'
st_olav_36['Type of imaging'] = 'MR'

In [9]:
def covert_modality(modality):
    if not 'CT' in modality:
        return 'MR'
    else:
        return 'CT'

In [49]:
st_36 = st_olav_36[['subject-id', 'timepoint', 'Type of imaging']]

In [35]:
def define_hosp(subject_id):
    match subject_id[:7]:
        case 'sub-100':
            return 'Ullevål'
        case 'sub-221':
            return 'Bærum'
        case 'sub-400':
            return 'St. Olav'
        case 'sub-300':
            return 'Haukeland'
        case 'sub-410':
            return 'Ålesund'

In [52]:
merged_imaging_data = pd.concat([base_imaging_data, st_36])

In [37]:
base_imaging_data['Hospital'] = base_imaging_data['subject-id'].apply(define_hosp)

In [28]:
base_imaging_data['Type of imaging'] = base_imaging_data['modality'].apply(covert_modality)

In [41]:
base_imaging_data.loc[base_imaging_data['timepoint'] == '36-mon-follow-up', ['subject-id','timepoint','Hospital']]['Hospital'].unique()

array(['Bærum'], dtype=object)

In [39]:
base_imaging_data['Hospital'].unique()

array(['Ullevål', 'St. Olav', 'Bærum', 'Haukeland', None, 'Ålesund'],
      dtype=object)

In [53]:
freesurfer.head()

,Unnamed: 0,pno,time,hosp,wmh_vol_bianca_ml_t0,freesurfer_t0,freesurfer_t2,freesurfer_t3,subject-id,hosp-id,timepoint
0,0,1,0,St. Olavs Hospital,NaN,NaN,NaN,NaN,sub-400001,sub-400,ses-study-MR
1,1,1,1,St. Olavs Hospital,NaN,NaN,NaN,NaN,sub-400001,sub-400,3-mon-follow-up
2,2,1,2,St. Olavs Hospital,NaN,NaN,NaN,NaN,sub-400001,sub-400,18-mon-follow-up
3,3,2,0,St. Olavs Hospital,NaN,NaN,NaN,NaN,sub-400002,sub-400,ses-study-MR
4,4,2,1,St. Olavs Hospital,NaN,NaN,NaN,NaN,sub-400002,sub-400,3-mon-follow-up


In [54]:
free_df = freesurfer[['subject-id', 'timepoint', 'wmh_vol_bianca_ml_t0', 'freesurfer_t0', 'freesurfer_t2','freesurfer_t3']]

In [71]:
free_df.to_csv('fresesurfer_sorted.csv')

In [72]:
postprocess = pd.melt(free_df, id_vars=['subject-id', 'timepoint'], value_vars=['wmh_vol_bianca_ml_t0', 'freesurfer_t0', 'freesurfer_t2','freesurfer_t3'], var_name='Postprocessing type', value_name='Postprocessing done')

In [73]:
postprocess['Postprocessing'] = postprocess.loc[postprocess['Postprocessing done'].notna(), 'Postprocessing type']

In [109]:
def correct_timepoint(row):
    if not pd.isnull(row['Postprocessing']):
        if 't0' in row['Postprocessing']:
            return 'ses-study-MR'
        elif 't2' in row['Postprocessing']:
            return '18-mon-follow-up'
        elif 't3' in row['Postprocessing']:
            return '36-mon-follow-up'

In [113]:
postprocess.loc[postprocess['Postprocessing'].notna(), ['Postprocessing', 'timepoint']]

,Postprocessing,timepoint
66,wmh_vol_bianca_ml_t0,ses-study-MR
67,wmh_vol_bianca_ml_t0,ses-study-MR
68,wmh_vol_bianca_ml_t0,ses-study-MR
101,wmh_vol_bianca_ml_t0,ses-study-MR
102,wmh_vol_bianca_ml_t0,ses-study-MR
...,...,...
10545,freesurfer_t3,36-mon-follow-up
10597,freesurfer_t3,36-mon-follow-up
10598,freesurfer_t3,36-mon-follow-up
10599,freesurfer_t3,36-mon-follow-up


In [110]:
postprocess['timepoint'] = postprocess.apply(lambda row: correct_timepoint(row), axis=1)

In [120]:
freesurfer_cleaned_results = postprocess.loc[postprocess['Postprocessing'].notna(), ['subject-id','timepoint','Postprocessing']].drop_duplicates().head()

In [121]:
freesurfer_cleaned_results

,subject-id,timepoint,Postprocessing
66,sub-400022,ses-study-MR,wmh_vol_bianca_ml_t0
101,sub-400033,ses-study-MR,wmh_vol_bianca_ml_t0
112,sub-400036,ses-study-MR,wmh_vol_bianca_ml_t0
119,sub-400038,ses-study-MR,wmh_vol_bianca_ml_t0
130,sub-400042,ses-study-MR,wmh_vol_bianca_ml_t0


In [124]:
base_imaging_data = pd.concat([base_imaging_data, freesurfer_cleaned_results])

In [125]:
base_imaging_data.to_csv('merged_imaging_data.csv', index=False)